ЗАДАНИЯ С БАЗОЙ ДАННЫХ:
1. Вытащите данные и найдите средний возраст клиентов из Минска
2. Выведите список всех клиентов с балансом больше 1000
3. Подсчитайте количество транзакций по каждому типу
4. Определите клиентов с наибольшими расходами (отрицательные суммы транзакций)
5. Топ-3 клиента по количеству транзакций
6. Выявление подозрительных операций, операции > n% от баланса

In [ ]:
import pandas as pd
import pandasql as ps
import sqlite3

with  sqlite3.connect('database.db') as conn:
    df_clients = pd.read_sql('select * from clients', conn)
    df_balance = pd.read_sql('select * from balance', conn)
    df_transaction = pd.read_sql('select * from transactions', conn)
df_clients

,client_id,name,age,city,account_number
0,1,Василий,34,Минск,1001
1,2,Екатерина,28,Москва,1002
2,3,Мария,45,Гродно,1003
3,4,Михаил,37,Витебск,1004
4,5,Евгения,29,Минск,1005


1. Вытащите данные и найдите средний возраст клиентов из Минска

In [ ]:
# query = ('select avg(age) from df_clients where city = "Минск"')
# result = ps.sqldf(query)

clients_minsk  = df_clients.query('city == "Минск"')
clients_minsk['age'].mean()



np.float64(31.5)

Выведите список всех клиентов с балансом больше 1000

In [12]:
# query = ('select * from df_balance where amount > 1000')
# result = ps.sqldf(query)
# result

balance = df_balance.query('amount > 1000')
balance


,client_id,amount
1,2,1500
3,4,2000
4,5,1200


Подсчитайте количество транзакций по каждому типу

In [17]:
# query = ('select type, count(*) from df_transaction group by type')
# result = ps.sqldf(query)
# result

df_transaction.groupby('type')['type'].count()

type
deposit       5
withdrawal    5
Name: type, dtype: int64

Определите клиентов с наибольшими расходами (отрицательные суммы транзакций)

In [ ]:
# query = ('select client_id, min(amount) from df_transaction where amount < 0')
# result =ps.sqldf(query)
# result

df_transaction.query('amount < 0').min()

transaction_id                 2
client_id                      1
date                  2025-03-02
amount                       -70
type                  withdrawal
description       ATM withdrawal
dtype: object

Топ-3 клиента по количеству транзакций

In [48]:
query = ('select client_id, count(*) as count from df_transaction group by client_id order by count DESC limit 3')
result = ps.sqldf(query)
result


df_transaction.groupby('client_id').count().sort_values(by='transaction_id', ascending=False)['transaction_id'].head(3)

client_id
1    2
2    2
3    2
Name: transaction_id, dtype: int64

6. Выявление подозрительных операций, операции > n% от баланса

In [81]:
total_df = df_balance.merge(df_transaction, how='inner', on='client_id')
total_df

,client_id,amount_x,transaction_id,date,amount_y,type,description
0,1,1000,1,2025-03-01,150,deposit,Salary
1,1,1000,2,2025-03-02,-50,withdrawal,ATM withdrawal
2,2,1500,3,2025-03-01,200,deposit,Freelance payment
3,2,1500,4,2025-03-03,-30,withdrawal,Grocery store
4,3,800,5,2025-03-02,100,deposit,Transfer from friend
5,3,800,6,2025-03-04,-20,withdrawal,Coffee shop
6,4,2000,7,2025-03-03,300,deposit,Bonus
7,4,2000,8,2025-03-05,-70,withdrawal,Restaurant
8,5,1200,9,2025-03-04,250,deposit,Investment return
9,5,1200,10,2025-03-06,-40,withdrawal,Shopping


In [92]:
n = 10

total_df['threshhold'] = total_df['amount_x'] * (n/100) 
# total_df.groupby('client_id')
total_df


,client_id,amount_x,transaction_id,date,amount_y,type,description,threshhold,is_suspicious
0,1,1000,1,2025-03-01,150,deposit,Salary,100.0,False
1,1,1000,2,2025-03-02,-50,withdrawal,ATM withdrawal,100.0,False
2,2,1500,3,2025-03-01,200,deposit,Freelance payment,150.0,False
3,2,1500,4,2025-03-03,-30,withdrawal,Grocery store,150.0,False
4,3,800,5,2025-03-02,100,deposit,Transfer from friend,80.0,False
5,3,800,6,2025-03-04,-20,withdrawal,Coffee shop,80.0,False
6,4,2000,7,2025-03-03,300,deposit,Bonus,200.0,False
7,4,2000,8,2025-03-05,-70,withdrawal,Restaurant,200.0,False
8,5,1200,9,2025-03-04,250,deposit,Investment return,120.0,False
9,5,1200,10,2025-03-06,-40,withdrawal,Shopping,120.0,False


In [93]:
total_df['is_suspicious'] = total_df['amount_y'].abs() > total_df['threshhold']
suspicious_operations = total_df[total_df['is_suspicious']]
suspicious_operations





,client_id,amount_x,transaction_id,date,amount_y,type,description,threshhold,is_suspicious
0,1,1000,1,2025-03-01,150,deposit,Salary,100.0,True
2,2,1500,3,2025-03-01,200,deposit,Freelance payment,150.0,True
4,3,800,5,2025-03-02,100,deposit,Transfer from friend,80.0,True
6,4,2000,7,2025-03-03,300,deposit,Bonus,200.0,True
8,5,1200,9,2025-03-04,250,deposit,Investment return,120.0,True
